In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
from geopy.extra.rate_limiter import RateLimiter
import time
import math

In [3]:
###set working directory
os.chdir('insert directory here')

#import lat and long data, registration data,
data = pd.read_csv(os.getcwd() + "/footprint_cost/ICLR/raw_data/latitude_and_longitude_ICLR.csv")

#Import cost data per distance, cost and footprint for hosp data, conference city coordinates
air_cost = pd.read_csv(os.getcwd() + "/footprint_cost/ICLR/raw_data/air_cost.csv")

In [4]:
####Define dataframe of conference cities and get coordinates with geolocator

#Open API
geolocator = Nominatim(user_agent="GoogleV3")
#Create dataframe of conferences cities and years
conf_city=pd.DataFrame([[2020,'Addis Ababa, Ethiopia'],[2019,'New Orleans, Louisiana'],[2018,'Vancouver, Canada']],columns=['Year','City'])
conf_city['Dest_Latitude']=0
conf_city['Dest_Longitude']=0

#cycle through rows and get lat and long
for row in conf_city.index:
    location = geolocator.geocode(conf_city.loc[row,'City'])
    if location is not None:
        conf_city.loc[row,'Dest_Latitude']=location.latitude
        conf_city.loc[row,'Dest_Longitude']=location.longitude
        
#Save dataframe to csv to be imported for later runs
conf_city.to_csv(os.getcwd() + "/footprint_cost/POM/raw_data/conference_city_POM.csv", index=False)


In [5]:
#Short-haul flight CO2 Emissions Calculations parameters
S_sh=153.51
PLF_sh=0.82
DC_sh=95
#(1-CF)
CF_sh=0.93
CW_econ_sh=0.96
CW_bus_sh=1.26
CW_1st_sh=2.4
EF_sh=3.15
P_sh=0.54
M_sh=2
AF_sh=0.00038
A_sh=11.68
a_sh=0
b_sh=2.714
c_sh=1166.52

#Long-haul flight CO2 Emissions Calculations parameters
S_lh=280.21
PLF_lh=0.82
DC_lh=95
#(1-CF)
CF_lh=0.74
CW_econ_lh=0.8
CW_bus_lh=1.54
CW_1st_lh=2.4
EF_lh=3.15
P_lh=0.54
M_lh=2
AF_lh=0.00038
A_lh=11.68
a_lh=0.0001
b_lh=7.104
c_lh=5044.93


In [6]:
conf_city

,Year,City,Dest_Latitude,Dest_Longitude
0,2020,"Addis Ababa, Ethiopia",9.010793,38.761252
1,2019,"New Orleans, Louisiana",29.949932,-90.070116
2,2018,"Vancouver, Canada",49.260872,-123.113953


In [7]:
#Define GHG emissions in CO2 equivalents for 1500 and 2500 km to be used for interpolating values for 1500<km<2500
co2_1500 = (a_sh*(1500**2) + b_sh*1500 + c_sh)/(S_sh * PLF_sh)*CF_sh*CW_econ_sh*(EF_sh*M_sh+P_sh)+AF_sh*1500+A_sh
co2_2500 = (a_lh*(2500**2) + b_lh*2500 + c_lh)/(S_lh * PLF_lh)*CF_lh*CW_econ_lh*(EF_lh*M_lh+P_lh)+AF_lh*2500+A_lh
#Cycle through rows in full list of registration ID coordinates
data['Local Attendee']=0
for row in data.index:
    print(row)
    
    #Separate coordinates for conference location
    conf_location=conf_city.loc[conf_city['Year']==data.loc[row,'Year']]
    #fill data column with conference location, latitutde and longitude
    data.loc[row,'Conference Location']=list(conf_location['City'])[0]
    data.loc[row,'Destination_Latitude']=float(conf_location['Dest_Latitude'])
    data.loc[row,'Destination_Longitude']=float(conf_location['Dest_Longitude'])
    
    #Define origin coordinates of attendee
    origin = (data.loc[row,'Origin_Latitude'],data.loc[row,'Origin_Longitude'])
    #Define conference destination coordinates
    destination = (data.loc[row,'Destination_Latitude'],data.loc[row,'Destination_Longitude'])

    #get great circle distance
    if sum(origin)!=0 and math.isnan(origin[0])==False and math.isnan(origin[1])==False:
        dist=great_circle(origin,destination).kilometers
        data.loc[row,'Travel Distance (round trip km)']=dist*2
        #define x as GCD + 95 (km) distance correction to account for inefficiencies
        x=dist + 95
        
        #If attendee is not from conference city
        if dist>100:
            #If short haul flight
            if x<1500:
                #Myclimate.org ghg calc
                co2_emissions = (a_sh*(x**2) + b_sh*x + c_sh)/(S_sh * PLF_sh)*CF_sh*CW_econ_sh*(EF_sh*M_sh+P_sh)+AF_sh*x+A_sh
            #If long haul flight
            elif x>2500:
                #Myclimate.org ghg calc
                co2_emissions = (a_lh*(x**2) + b_lh*x + c_lh)/(S_lh * PLF_lh)*CF_lh*CW_econ_lh*(EF_lh*M_lh+P_lh)+AF_lh*x+A_lh
            #If in between
            else:
                #Interpolate value between 1500 and 2500 km
                co2_emissions = (co2_2500-co2_1500)/(2500-1500)*(x-1500)+co2_1500
                
            #Add travel emissions to data
            data.loc[row,'Travel CO2 (tonnes)']= co2_emissions/1000*2
    
    #If attendee is from conference city, set travel distance and CO2, hotel cost and CO2, Food cost to 0 
    if dist<=100:
        data.loc[row,'Travel CO2 (tonnes)']=0
        data['Local Attendee']=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272


In [8]:
#Cycle through all attendees
for row in data.index:
    #define x as GCD for cost calculations
    x=data.loc[row,'Travel Distance (round trip km)']/2
    #check if the attendee doesn't live near the conference city
    if x > 100:
        #If distance is in each bin, multiply distance by corresponding cost per km
        if x >= air_cost.loc[0,'km_low'] and x <= air_cost.loc[0,'km_high']:
            data.loc[row,'Travel Cost (USD)'] = 1.5 * x * air_cost.loc[0,'Price_per_km']
        elif x > air_cost.loc[1,'km_low'] and x <= air_cost.loc[1,'km_high']:
            data.loc[row,'Travel Cost (USD)'] = 1.5 * x * air_cost.loc[1,'Price_per_km']
        elif x > air_cost.loc[2,'km_low'] and x <= air_cost.loc[2,'km_high']:
            data.loc[row,'Travel Cost (USD)'] = 1.5 * x * air_cost.loc[2,'Price_per_km']
        elif x > air_cost.loc[3,'km_low']:
            data.loc[row,'Travel Cost (USD)'] = 1.5 * x * air_cost.loc[3,'Price_per_km']
    #if the attendee lives near the conference city, set travel cost to 0
    else:
        data.loc[row,'Travel Cost (USD)'] = 0
       

In [9]:
#export data
data.to_csv(os.getcwd() + '/footprint_cost/ICLR/footprint_calcs_output/attendee_footprint_cost_ICLR_1.5_sens.csv', index=False)